# CS572 Project
* We selected the paper: {}
* Based on the data and model of given paper, we are planning to make an extension of LSTM that predicts {}.

## Preparing

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import time
# from matplotlib import pyplot as plt

data_path_train = 'Data/Training' # add path
data_path_test = 'Data/Testing'

input_size = 8
hidden_size = 128
num_layers = 3
num_epochs = 2000
batch_size = 32
learning_rate = 0.001
weight_decay=0.00001

Check if your computer is prepared to run pytorch model with CUDA.

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # x -> batch_size, seq, input_size
        self.fc = nn.Linear(hidden_size, 1)
        
#     def makehc_zero(self, batch_size):
#         self.hn = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
#         self.cn = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
#         print("clear")
        
    
    def forward(self, x):
#         if self.training == True:
#             # Set initial hidden and cell states 
#             self.makehc_zero(x.size(0))

        
        # Forward propagate LSTM
#         out, (self.hn, self.cn) = self.lstm(x, (self.hn, self.cn))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        out, (self.hm, self.cn) = self.lstm(x,(h0,c0))

        out_original = out
        out_original_fc = self.fc(out_original) # out_original : Tensor of shape (batch_size, seq_length, 1)
        
#         pred_y = self.fc(out[-1].view(x.size(1),-1)) # pred_y : tensor of shape (40,1), taking the last section of a batch
#         print("This is x.size(1):", x.size(1))
#         print("This is out[-1] :", out[-1].shape)
#         print(out[-1])
#         print("This is out[-1].view(x.size(1),-1):", out[-1].view(x.size(1),-1).shape)
#         print(out[-1].view(x.size(1),-1))
#         print("This is pred_y : ", pred_y.shape)
#         print(pred_y)
        

#         out = out[:, -1, :]
#         print("This is output right before linear : ", out.shape)
#         out = self.fc(out)
#         print("This is output after linear : ", out.shape)
#         print(out)
        
        return out_original_fc

Data Loading

In [36]:
# Training and Testing: To train fingertip forces from input
# data, a three-layer BT-LSTM with 128 hidden units is used.
# Sequence time length of BT-LSTM was set to T = 40. The final
# output of the BT-LSTM layer is fed into the final fully-connected
# layer to obtain the one-dimensional output. The mean squared
# error (MSE) function is used to measure the loss between the
# predicted and the ground truth contact force. During training,
# the number of epoch was 2000, and the mini batch size was 32.
# Adam optimizer was used with 0.001 learning rate and 10−5
# weight decay
lstm = LSTM(input_size, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=weight_decay)

Load cell if needed

In [ ]:
lstm.load_state_dict(torch.load('model.ckpt'))

Train model

In [37]:
for path_idx in os.listdir(data_path_train):
    cur_path = os.path.join(data_path_train, path_idx)
    print(cur_path)
    input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
    # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
    inputs = torch.Tensor(input_file)
#     print(inputs[0])
    
    inputs = inputs[:len(inputs)-len(inputs)%40]
#     labels = inputs[:, [5]]
# #     print(labels.shape)
#     labels = labels.view([-1, 40, 1])
    
    inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
    inputs = inputs.view([-1,40,9])
    
    train_loader = torch.utils.data.DataLoader(dataset=inputs,
                                           batch_size=batch_size, 
                                           shuffle=True)
    # Train the model
    start = time.time()
    # for epoch in range(num_epochs):
    for epoch in range(100):    
        for i, inputs in enumerate(train_loader):
            inputs, labels = torch.split(inputs, [8, 1], 2)
#             print("inputs shape : ", inputs.shape)
            inputs = inputs.to(device)
    #         labels = labels[:,[39],:].view([-1,1])
    #         labels = labels[-1,:,:].view([-1,1])
            labels = labels.to(device)
    #         print("labels shape : ", labels.shape)


            # Forward pass
            outputs = lstm(inputs)

    #         print(labels.shape)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            print('time :', time.time() - start)
    print("training complete")

Data/Training\190722_tension_test_space_1_trial_2.csv
Epoch [10/2000], Loss: 0.5635
time : 1.1653928756713867
Epoch [20/2000], Loss: 0.5122
time : 2.2076048851013184
Epoch [30/2000], Loss: 0.7140
time : 3.245828151702881
Epoch [40/2000], Loss: 0.5939
time : 4.2925379276275635
Epoch [50/2000], Loss: 0.4798
time : 5.321784734725952
Epoch [60/2000], Loss: 0.6846
time : 6.365991592407227
Epoch [70/2000], Loss: 1.6757
time : 7.4082043170928955
Epoch [80/2000], Loss: 0.3441
time : 8.425988912582397
Epoch [90/2000], Loss: 0.7402
time : 9.461220502853394
Epoch [100/2000], Loss: 0.6738
time : 10.477445125579834
training complete
Data/Training\190722_tension_test_space_1_trial_3.csv
Epoch [10/2000], Loss: 2.3629
time : 5.750633955001831
Epoch [20/2000], Loss: 0.7375
time : 11.424978494644165
Epoch [30/2000], Loss: 2.6981
time : 17.084346532821655
Epoch [40/2000], Loss: 0.7175
time : 22.77531099319458
Epoch [50/2000], Loss: 0.1489
time : 28.423710346221924
Epoch [60/2000], Loss: 1.9131
time : 34.

Epoch [10/2000], Loss: 0.5877
time : 5.419503688812256
Epoch [20/2000], Loss: 0.1502
time : 10.835540056228638
Epoch [30/2000], Loss: 0.3031
time : 16.230143070220947
Epoch [40/2000], Loss: 0.1629
time : 21.64465880393982
Epoch [50/2000], Loss: 0.1743
time : 27.028257369995117
Epoch [60/2000], Loss: 0.1178
time : 32.40688610076904
Epoch [70/2000], Loss: 0.2076
time : 37.777551889419556
Epoch [80/2000], Loss: 0.1637
time : 43.09133744239807
Epoch [90/2000], Loss: 0.2175
time : 48.43105435371399
Epoch [100/2000], Loss: 0.4858
time : 53.7667977809906
training complete
Data/Training\test11thick_1.csv
Epoch [10/2000], Loss: 0.8100
time : 1.861527681350708
Epoch [20/2000], Loss: 1.0716
time : 3.6976571083068848
Epoch [30/2000], Loss: 0.5253
time : 5.5292649269104
Epoch [40/2000], Loss: 1.5593
time : 7.345406770706177
Epoch [50/2000], Loss: 0.6014
time : 9.162546157836914
Epoch [60/2000], Loss: 0.9765
time : 10.953264236450195
Epoch [70/2000], Loss: 0.3933
time : 12.803316354751587
Epoch [80/

Test model

In [38]:
start = time.time()
Expectation = []
Truth = []

with torch.no_grad():
    for path_idx in os.listdir(data_path_train):
        cur_path = os.path.join(data_path_train, path_idx)
        print(cur_path)
        input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
        # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
               
        inputs = torch.Tensor(input_file)
        inputs = inputs[:len(inputs)-len(inputs)%40]
        inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
        inputs = inputs.view([-1,40,9])
        
        test_loader = torch.utils.data.DataLoader(dataset=inputs,
                                       batch_size=batch_size, 
                                       shuffle=False)
        
        for inputs in test_loader:

            inputs, labels = torch.split(inputs, [8,1], 2)
            inputs = inputs.to(device)
            labels = labels[:, : ,:].view([-1,1])
            outputs = lstm(inputs)
            outputs = outputs[:,:,:].view([-1,1])
            
            labels_list = labels.tolist()
            outputs_list = outputs.tolist()
            
            zip_lists = zip(labels_list, outputs_list)
            for labels, outputs in zip_lists:
                Truth.append(labels)
                Expectation.append(outputs)

#             for i in range(inputs.size(1)):
#                 outputs = lstm(inputs[:,i,:].view([inputs.size(0),1,8]))
#                 total_outputs += outputs.cpu()           
            
        break

    print("This is Expectation size", len(Expectation))
    rmse = mean_squared_error(Expectation, Truth, squared = False)
    mae = mean_absolute_error(Expectation, Truth)
    
    print('Test Accuracy of the model RMSE: {}'.format(rmse))
    print('Test Accuracy of the model MAE: {}'.format(mae)) 
    print('time :', time.time() - start)
    


Data/Training\190722_tension_test_space_1_trial_2.csv
This is Expectation size 14040
Test Accuracy of the model RMSE: 2.2575374285401213
Test Accuracy of the model MAE: 1.385511378384994
time : 0.26429200172424316


Save model

In [32]:
torch.save(lstm.state_dict(), 'model.ckpt_Hochang')
print("save complete")

save complete


In [29]:
plt.plot(Expectation)
plt.plot(Truth)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.savefig('fig1.png', dpi=300)

In [ ]:
print(total_labels)